In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
marketing_campaign = pd.read_csv("dataset/marketing_campaign_clean.csv")
marketing_campaign.shape

(2147, 13)

In [3]:
marketing_campaign.head()

,ID,Year_Birth,Education,Income,Kidhome,MntWines,Customer_Age,Customer_Generation,Customer_Age_Group,Customer_Age_Group2,Education2,Kidhome2,Amount_Group
0,5524,1957,Undergraduate,58138.0,0,635,62,Baby Boomers & Older,Adults,Seniors,Undergraduate & Master,0,301-1000
1,2174,1954,Undergraduate,46344.0,1,11,65,Baby Boomers & Older,Seniors,Seniors,Undergraduate & Master,1,0-40
2,4141,1965,Undergraduate,71613.0,0,426,54,Generation X,Adults,Adults,Undergraduate & Master,0,301-1000
3,6182,1984,Undergraduate,26646.0,1,11,35,Millennials & Generation Z,Adults,Adults,Undergraduate & Master,1,0-40
4,5324,1981,PhD,58293.0,1,173,38,Millennials & Generation Z,Adults,Adults,PhD,1,101-300


In [4]:
def get_age_group(age):
    if age < 0:
        raise Exception("Negative age is invalid")
    elif age <= 60:
        return "Adults"
    else:
        return "Seniors"
    
def get_generation(year):
    if year <= 1955:
        return "Baby Boomers & Older"
    elif year <= 1965:
        return "Generation X"
    elif year <= 2012:
        return "Millennials & Generation Z"
    else:
        return np.nan


In [5]:
def annualize_data(data, start_year, end_year, pivot_column, pct=True):
    def pivot_data(df):
        for col in df.columns:
            v = df.loc[~df[col].isna(), col].values[0]
            df[col] = df[col].fillna(v)
        df = df.drop_duplicates()
        return df
        
    df = None
    for year in range(start_year, end_year+1):
        tmp = data.copy()
        tmp["Customer_Age"] = year - tmp["Year_Birth"]
        # Since the drinking age in the US is 21, we get rid of ineligible customers
        tmp = tmp[tmp["Customer_Age"] >= 21]
        # We also get rid of dead people
        tmp = tmp[tmp["Customer_Age"] < 100]
        
        tmp["Customer_Age_Group"] = tmp["Customer_Age"].apply(lambda x: get_age_group(int(x)))
        tmp["Customer_Generation"] = tmp["Year_Birth"].apply(lambda x: get_generation(int(x)))
        tmp["Has_Small_Children"] = tmp["Kidhome"].apply(lambda x: "Small_Children" if x==1 else "No_Small_Children")
        
        # Aggregate data for each year
        numerical_cols = ["Income"]
        categorical_cols = ["Customer_Age_Group", "Has_Small_Children"]
        
        if pivot_column in numerical_cols:
            numerical_cols.remove(pivot_column)
        if pivot_column in categorical_cols:
            categorical_cols.remove(pivot_column)
        
        merged_agg = None
        
        for col in numerical_cols + categorical_cols:
            agg = tmp[[pivot_column, "ID", col]].groupby([pivot_column, col], as_index=False).count()
            
            # Add year column
            agg["Year"] = year
            # Pivot the dataframe
            if col in categorical_cols:
                agg_pivoted = agg.pivot_table(index='Year', columns=[pivot_column, col], values='ID', aggfunc='sum').reset_index(drop=True)
                # Flatten the multi-index columns
                agg_pivoted.columns = [f"{col[0]}_Num_{col[1]}".strip() for col in agg_pivoted.columns.values]
                # Add back year
                agg_pivoted["Year"] = year
                if pct:
                    agg['pct'] = agg.apply(lambda x: x['ID'] / agg.loc[agg[pivot_column] == x[pivot_column], 'ID'].sum() * 100, axis=1)
                    agg_pivoted_pct = agg.pivot_table(index='Year', columns=[pivot_column, col], values='pct', aggfunc='sum').reset_index(drop=True)
                    agg_pivoted_pct.columns = [f"{col[0]}_Pct_{col[1]}".strip() for col in agg_pivoted_pct.columns.values]
                    # Add back year
                    agg_pivoted_pct["Year"] = year
                    # Merge
                    agg_pivoted = pd.merge(agg_pivoted, agg_pivoted_pct, on="Year", how="inner")
            else:
                agg_pivoted = agg.pivot_table(index='Year', columns=pivot_column, values=[col], aggfunc='mean').reset_index(drop=True)
                # Flatten the multi-index columns
                agg_pivoted.columns = [f"{col[1]}_Avg_{col[0]}".strip() for col in agg_pivoted.columns.values]
                # Add back year
                agg_pivoted["Year"] = year
            
            # Reset index to align with desired output
            agg_pivoted.reset_index(drop=True, inplace=True)
            
            if merged_agg is None:
                merged_agg = agg_pivoted
            else:
                merged_agg = pd.merge(merged_agg, agg_pivoted, on="Year", how="inner")
        
        if df is None:
            df = merged_agg
        else:
            df = pd.concat([df, merged_agg])
    
    df = df.sort_values(by=["Year"])
    df = df.reset_index(drop=True)
    
    return df

In [6]:
marketing_campaign_annualized = annualize_data(marketing_campaign, 1999, 2021, pivot_column="Customer_Age_Group")

In [7]:
marketing_campaign_annualized = marketing_campaign_annualized.fillna(0)

In [8]:
marketing_campaign_annualized.head()

,Adults_Avg_Income,Seniors_Avg_Income,Year,Adults_Num_No_Small_Children,Adults_Num_Small_Children,Seniors_Num_Small_Children,Adults_Pct_No_Small_Children,Adults_Pct_Small_Children,Seniors_Pct_Small_Children,Seniors_Num_No_Small_Children,Seniors_Pct_No_Small_Children
0,53844.015043,36640.0,1999,1084,621,1.0,63.577713,36.422287,100.0,0.0,0.0
1,53715.629063,0.0,2000,1106,646,0.0,63.127854,36.872146,0.0,0.0,0.0
2,53679.554863,51141.0,2001,1121,667,0.0,62.695749,37.304251,0.0,1.0,100.0
3,53529.965791,72084.0,2002,1130,692,0.0,62.019759,37.980241,0.0,2.0,100.0
4,53391.429169,72084.0,2003,1156,708,0.0,62.017167,37.982833,0.0,2.0,100.0
